In [3]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

2025-06-26 10:31:45.520294: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-26 10:31:45.522009: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 10:31:45.529325: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 10:31:45.552770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750914105.588724   17511 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750914105.59

In [4]:
#loading model
model = load_model('model.h5')
#gender
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

#encoder 
with open('oh_encoder_geo.pkl','rb') as file:
    oh_encoder_geo = pickle.load(file)

#scaler 
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

2025-06-26 10:31:51.659805: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
#example input data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [6]:
#onehot encode Geography column
geo_encoded = oh_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=oh_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/home/morty/Documents/ANN Classification/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
input_data['Gender'] = label_encoder_gender.transform(input_data.Gender)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
input_data = pd.concat([input_data.drop(['Geography'],axis=1),geo_encoded_df],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
#scaling
input_data_scaled = scaler.transform(input_data)
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
#prediction whether he is going to leave the bank or not
prediction = model.predict(input_data_scaled)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step


array([[0.0241704]], dtype=float32)

In [ ]:
# array([[0.0241704]] is the probability of leaving 
#therefore if it is greater than 0.5 he is going to leave else not
